In [ ]:
!pip3 install kobert-transformers
!pip install transformers
!pip install accelerate -U
!pip install torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
import torch
from kobert_transformers import get_kobert_model, get_tokenizer
from transformers import TrainingArguments, Trainer
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
file_path = '/content/drive/MyDrive/pretzel2/total_data.xlsx'  # 데이터 파일 경로
df = pd.read_excel(file_path)

In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(df['content'].tolist(), df['emotion'].tolist(),
                                                                    test_size=0.2, random_state=42)

In [ ]:
# 3. 모델 및 토크나이저 로드
tokenizer = get_tokenizer()
model = get_kobert_model(num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# 4. 데이터셋 클래스 정의
class EmotionDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)  # 데이터셋의 길이 반환

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=64, return_tensors="pt")
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=64, return_tensors="pt")

In [ ]:
train_dataset = EmotionDataset(train_encodings, train_labels)
val_dataset = EmotionDataset(val_encodings, val_labels)

In [ ]:
# 5. 훈련 인자 설정 및 훈련
training_args = TrainingArguments(
    output_dir='./output',
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=10,
    learning_rate=2e-5, # 학습률
    lr_scheduler_type='linear', # 학습률을 선형적으로 감소
    weight_decay=0.01, # L2 규제 가중치 감쇠
    logging_dir='./logs',
    logging_steps=200,
    save_steps=1000,
    eval_strategy="epoch",
    save_strategy="epoch",
    warmup_ratio=0.1,
    max_grad_norm=1.0,
    seed=42
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=lambda pred: {"accuracy": accuracy_score(pred.label_ids, pred.predictions.argmax(axis=1))}
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.018800,0.943783,0.600750
2,0.759900,0.708096,0.695892
3,0.669300,0.640324,0.722823
4,0.620900,0.622109,0.732276
5,0.601700,0.602522,0.736092
6,0.576400,0.602520,0.737628
7,0.560000,0.602438,0.739983
8,0.537900,0.612539,0.741215
9,0.526400,0.616200,0.740961
10,0.521000,0.628724,0.740802


TrainOutput(global_step=98450, training_loss=0.6762677075960964, metrics={'train_runtime': 16427.5826, 'train_samples_per_second': 383.533, 'train_steps_per_second': 5.993, 'total_flos': 2.0722263986966016e+17, 'train_loss': 0.6762677075960964, 'epoch': 10.0})

In [ ]:
# 6. 평가 및 저장
eval_result = trainer.evaluate()
print("Evaluation result:", eval_result)

Evaluation result: {'eval_loss': 0.628723680973053, 'eval_accuracy': 0.7408023464729895, 'eval_runtime': 121.7508, 'eval_samples_per_second': 1293.733, 'eval_steps_per_second': 20.222, 'epoch': 10.0}


In [ ]:
model_path = "/content/drive/MyDrive/pretzel2/kobert_emotion_classification3"
trainer.save_model(model_path)
print(f"Trained model saved to {model_path}")

Trained model saved to /content/drive/MyDrive/pretzel2/kobert_emotion_classification3
